In [38]:
# Cài thư viện nếu cần
!pip install pandas scikit-learn matplotlib seaborn --quiet


#Bước 1: Tải và đọc dữ liệu

In [39]:
import pandas as pd

# Tải file từ Google Drive hoặc upload trực tiếp
from google.colab import files
uploaded = files.upload()  # Chọn annonimized.csv, th-public.csv,...

# Đọc dữ liệu nộp bài
df = pd.read_csv("annonimized.csv")
df.head()


Saving annonimized.csv to annonimized (1).csv
Saving th-public.csv to th-public (1).csv
Saving ck-public.csv to ck-public (1).csv
Saving qt-public.csv to qt-public (1).csv


,"concat('it001',`assignment_id`)","concat('it001',`problem_id`)","concat('it001', username)",is_final,status,pre_score,coefficient,"concat('it001',`language_id`)",created_at,updated_at,judgement
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."


# Bước 2: Tiền xử lý dữ liệu


In [40]:
import numpy as np

# Chuyển datetime
df['created_at'] = pd.to_datetime(df['created_at'], format='%m-%d %H:%M:%S', errors='coerce')
df['updated_at'] = pd.to_datetime(df['updated_at'], format='%m-%d %H:%M:%S', errors='coerce')


# Trích đặc trưng thời gian
df['submit_hour'] = df['created_at'].dt.hour
df['submit_weekday'] = df['created_at'].dt.weekday
df['delay'] = (df['updated_at'] - df['created_at']).dt.total_seconds()

# Use raw 'concat('it001', username)' as 'mssv'
df['mssv'] = df["concat('it001', username)"]

# Encode categorical
df['assignment_id'] = df["concat('it001',`assignment_id`)"].astype('category').cat.codes
df['problem_id'] = df["concat('it001',`problem_id`)"].astype('category').cat.codes

# Đảm bảo số liệu
df['percent_testcase_passed'] = df['pre_score'].fillna(0) / 100 # Assuming pre_score is out of 10000
df['delay_weight'] = df['coefficient'].fillna(100) / 100 # Assuming coefficient is out of 100
df['has_score'] = df['status'] == 'SCORE'
df['is_passed'] = df['percent_testcase_passed'] == 100

# Preview
df[['mssv', 'assignment_id', 'problem_id', 'percent_testcase_passed', 'submit_hour', 'submit_weekday', 'delay', 'delay_weight', 'has_score', 'is_passed']].head()

,mssv,assignment_id,problem_id,percent_testcase_passed,submit_hour,submit_weekday,delay,delay_weight,has_score,is_passed
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,0.0,8.0,1.0,294.0,1.0,True,False
1,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,0.0,8.0,1.0,10.0,1.0,True,False
2,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,100.0,8.0,1.0,9.0,1.0,True,True
3,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,335,100.0,8.0,1.0,9.0,1.0,True,True
4,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,212,100.0,9.0,1.0,10.0,1.0,True,True


#Bước 3: Trích đặc trưng cho từng sinh viên



In [41]:
# Gom theo mssv để tạo 1 dòng/mssv
features = df.groupby('mssv').agg({
    'percent_testcase_passed': ['mean', 'std', 'max'],
    'delay_weight': 'mean',
    'submit_hour': 'mean',
    'submit_weekday': 'nunique',
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'delay': 'mean',
    'has_score': 'sum',
    'is_passed': 'sum'
})

# Đổi tên cột
features.columns = ['_'.join(col) for col in features.columns]
features.reset_index(inplace=True)

# Preview
features.head()

,mssv,percent_testcase_passed_mean,percent_testcase_passed_std,percent_testcase_passed_max,delay_weight_mean,submit_hour_mean,submit_weekday_nunique,assignment_id_nunique,problem_id_nunique,delay_mean,has_score_sum,is_passed_sum
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,55.041497,45.414358,100.0,1.000000,4.401361,7,7,46,8195.428571,119,54
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,54.885521,43.237609,100.0,1.000000,10.525097,7,9,78,-232304.548263,209,85
2,01122b3ef7e59b84189e65985305f575d6bdf83c,59.737538,41.699366,100.0,1.000000,10.964103,7,7,66,-103020.830769,175,59
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,59.527600,46.403021,100.0,1.000000,8.610000,7,4,47,1514.090000,76,52
4,013de369c439ab0ead8aa7da64423aa395a8be39,64.744486,44.096659,100.0,0.971963,8.598131,4,8,52,184.037383,94,59


#Bước 4: Chuẩn bị nhãn (label)

In [42]:
# Đọc điểm thực hành thật
df_score = pd.read_csv("th-public.csv")  # file này có 2 cột: mssv, diem_thuc_hanh

# Đổi tên cột 'hash' thành 'mssv' trong df_score
df_score = df_score.rename(columns={'hash': 'mssv'})

# Ghép vào feature
df_merged = pd.merge(features, df_score, on='mssv')
df_merged.head()

,mssv,percent_testcase_passed_mean,percent_testcase_passed_std,percent_testcase_passed_max,delay_weight_mean,submit_hour_mean,submit_weekday_nunique,assignment_id_nunique,problem_id_nunique,delay_mean,has_score_sum,is_passed_sum,TH
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,55.041497,45.414358,100.0,1.000000,4.401361,7,7,46,8195.428571,119,54,5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,54.885521,43.237609,100.0,1.000000,10.525097,7,9,78,-232304.548263,209,85,8.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,59.737538,41.699366,100.0,1.000000,10.964103,7,7,66,-103020.830769,175,59,7
3,013de369c439ab0ead8aa7da64423aa395a8be39,64.744486,44.096659,100.0,0.971963,8.598131,4,8,52,184.037383,94,59,10
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,52.400251,46.917106,100.0,1.000000,11.989950,7,9,90,-252938.427136,154,79,6


#Bước 5: Train/Test mô hình

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np

# Tách features và labels
X = df_merged.drop(columns=['mssv', 'TH'])
y = df_merged['TH']

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean of the training data
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean()) # Use mean from training data to avoid data leakage

# Convert y to numeric, coercing errors to NaN and then dropping NaN
y_train = pd.to_numeric(y_train, errors='coerce').dropna()
y_test = pd.to_numeric(y_test, errors='coerce').dropna()

# Ensure X_train and y_train have the same index after dropping NaNs in y_train
X_train = X_train.loc[y_train.index]
X_test = X_test.loc[y_test.index]


# Huấn luyện model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2:.4f}")

R^2 Score: 0.3750


#Bước 6: Dự đoán cho tất cả sinh viên

In [44]:
# Dự đoán cho tất cả sinh viên có trong annonimized.csv
features_all = df.groupby('mssv').agg({
    'percent_testcase_passed': ['mean', 'std', 'max'],
    'delay_weight': 'mean',
    'submit_hour': 'mean',
    'submit_weekday': 'nunique',
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'delay': 'mean',
    'has_score': 'sum',
    'is_passed': 'sum'
})
features_all.columns = ['_'.join(col) for col in features_all.columns]
features_all.reset_index(inplace=True)

# Dự đoán
X_all = features_all.drop(columns=['mssv'])
y_all_pred = model.predict(X_all)

# Tạo file submission
submission = pd.DataFrame({
    'mssv': features_all['mssv'],
    'diem_du_doan': y_all_pred
})

# Làm tròn nếu muốn
submission['diem_du_doan'] = submission['diem_du_doan'].clip(0, 10).round(2)
submission.head()

,mssv,diem_du_doan
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.72
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.32
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.61
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,6.64
4,013de369c439ab0ead8aa7da64423aa395a8be39,9.22


#Bước 7: Xuất file để nộp

In [46]:
# Xuất file
submission.to_csv("submission_thuc_hanh.csv", index=False,header=False)

# Tải về
files.download("submission_thuc_hanh.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>